In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dotenv import load_dotenv
load_dotenv()

import asyncio

from computer_use_demo.loop import sampling_loop, PROVIDER_TO_DEFAULT_MODEL_NAME, APIProvider, SYSTEM_PROMPT
import os

from playwright.async_api import async_playwright

In [ ]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
page = await browser.new_page(viewport={"width": 1024, "height": 768})

In [ ]:
messages = []

In [ ]:
from print_utils import noop
from conversation_utils import display_conversation_markdown


def print_diff(old_messages, new_messages):
    # old_messages_set = set(old_messages)
    messages_to_print = [
        message for message in new_messages if message not in old_messages
    ]
    display_conversation_markdown(messages_to_print)
    old_messages.extend(messages_to_print)
            
async def converse():
    old_messages = [*messages]
    await sampling_loop(
            model=PROVIDER_TO_DEFAULT_MODEL_NAME[APIProvider.ANTHROPIC],
            api_key=os.environ.get("ANTHROPIC_API_KEY"),
            provider=APIProvider.ANTHROPIC,
            messages=messages,
            system_prompt_suffix="",
            output_callback=lambda _: print_diff(old_messages, messages),
            tool_output_callback=lambda _, __: print_diff(old_messages, messages),
            api_response_callback=noop,
            page=page,
        )
    await asyncio.sleep(0.1)

In [ ]:
user_message = input()
if len(user_message) > 0:
    messages.append({
        "role": "user",
        "content": user_message
    })
    await converse()
    

In [ ]:
display_conversation_markdown(messages)

In [ ]:
await browser.close()
await playwright.stop()

In [ ]:
from conversation_utils import save_conversation

save_conversation(messages, "outputs/americanino_with_xpath.json")

In [ ]:
from dotenv import load_dotenv
load_dotenv()
from conversation_utils import load_conversation, replay_conversation

loaded = load_conversation("outputs/americanino_with_xpath.json")
loaded_playwright, browser, context, page, tools, tool_uses = await replay_conversation(
    loaded
)